In [1]:
import os

import numpy as np

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [14]:
TEST = False

table = 'movement_model_inference_m3_a3_v8'

table = table if not TEST else f'{table}_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data['log_prob'] = np.log(data['probability'])
data.head()

,_decision,_choice,_selected,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,...,sin_time,tag_key,log_odds,odds,probability,experiment_name,run_id,_train,_partition,log_prob
0,3,2,False,10.671942,18.879669,2.250513,-2.146755,68.192535,841d209ffffffff,841d20bffffffff,...,0.281733,229228,-8.069914,0.000313,0.001397,movement-model-m3-a3-v8,18771d53ba844e6678048b4166818d300ee21201f33b83...,True,0,-6.573275
1,24,12,False,11.802080,6.680935,1.140766,-3.104071,66.449379,841d201ffffffff,841d201ffffffff,...,-0.281733,229228,-8.172777,0.000282,0.001220,movement-model-m3-a3-v8,18771d53ba844e6678048b4166818d300ee21201f33b83...,True,0,-6.708687
2,21,18,False,10.813726,16.543697,1.402085,-0.279220,83.133400,841d201ffffffff,841d201ffffffff,...,-0.281733,229228,-6.285064,0.001864,0.008059,movement-model-m3-a3-v8,18771d53ba844e6678048b4166818d300ee21201f33b83...,True,0,-4.820973
3,9,5,False,10.523235,21.873289,2.707258,1.859337,42.236572,841d20bffffffff,841d20bffffffff,...,0.281733,229228,-5.531146,0.003961,0.014996,movement-model-m3-a3-v8,18771d53ba844e6678048b4166818d300ee21201f33b83...,True,0,-4.199997
4,12,2,False,10.547960,33.893909,2.348950,-2.746964,37.475468,841d20bffffffff,841d20bffffffff,...,-0.281733,229228,-4.813920,0.008116,0.035760,movement-model-m3-a3-v8,18771d53ba844e6678048b4166818d300ee21201f33b83...,True,0,-3.330926


In [15]:
final = (
    data[data['_selected']]
    .groupby(['_individual', '_train'])[['log_prob']]
    .mean().reset_index()
    .groupby('_train')[['log_prob']].mean()
)
final['GMP'] = np.exp(final['log_prob'])
final

,log_prob,GMP
_train,,
False,-1.336710,0.262709
True,-1.234926,0.290856
